# Machine Learning - Practicum 5
<b>Student:</b> Justin Klein<br>
<b>Nummer:</b> 1707815

In [1]:
import ssl
import tensorflow as tf
import tensorflow.keras.datasets.cifar10 as cifar
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import utils

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Data Analysis and Preparation
Eerst laden we de cifar10 data in en verdelen we de data die gebruikt wordt voor het trainen en testen in hun bijbehorende variabelen.

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context
data = cifar.load_data()

In [3]:
train_data = data[0]
test_data = data[1]

print(f"Length of train_data feature records: {len(train_data[0])}, and target records: {len(train_data[1])}")
print(f"Length of test_data feature records: {len(test_data[0])}, and target records: {len(test_data[1])}")

Length of train_data feature records: 50000, and target records: 50000
Length of test_data feature records: 10000, and target records: 10000


Nu verdelen we de features en targets in nieuwe variabelen zodat we het kunnen gebruiken in een CNN. Voor de targets gebruiken we one-hot encoding.

In [4]:
X_train_data = train_data[0]
X_test_data = test_data[0]

y_train_data = utils.to_categorical(train_data[1], 10)
y_test_data = utils.to_categorical(test_data[1], 10)

Nu zien we dat de data goed ingedeeld is in variabelen en dat de data de juiste input dimensies heeft voor het CNN.

In [5]:
print(f"Size of a picture in X_train is: {len(X_train_data[0])}x{len(X_train_data[0][0])}")
print(f"Size of a picture in X_test is: {len(X_test_data[0])}x{len(X_test_data[0][0])}")

Size of a picture in X_train is: 32x32
Size of a picture in X_test is: 32x32


## Modelling CNN
Nu kunnen we het convolutionele netwerk bouwen. Voor het netwerk heb ik eerst gekozen voor twee convolutionele layers met maxpools. Daarna komt nogmaals een convolutional layer en flattening. De rede waarom ik dit doe is omdat we hieruit een dimensie van 1024x1 krijgen (32x32 is 1024) wat we aan het FC netwerk kunnen geven.

In [1]:
model = Sequential()

# Convolution Layer x2 + Pools
model.add(Conv2D(filters=16, kernel_size=(3, 3), input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolution Layer + Flattening
model.add(Conv2D(filters=64, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(Flatten())

# FC Layers
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))


NameError: name 'Sequential' is not defined

Vervolgens ziet het netwerk er zo uit:

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 16)        448       
                                                                 
 activation (Activation)     (None, 30, 30, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        9280      
                                                                 
 activation_1 (Activation)   (None, 13, 13, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                    

Nu compilen we het netwerk, en omdat we met one-hot encodig werken gebruiken we 'categorical_crossentropy'.

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Training the Model
Nu we het model hebben opgesteld gaan we het model trainen met de trainingsdata.

In [9]:
model.fit(X_train_data, y_train_data, batch_size=128, epochs=50, verbose=1)

Epoch 1/40
391/391 [==============================] - 15s 38ms/step - loss: 2.0543 - accuracy: 0.2728
Epoch 2/40
391/391 [==============================] - 15s 40ms/step - loss: 1.4343 - accuracy: 0.4712
Epoch 3/40
391/391 [==============================] - 15s 39ms/step - loss: 1.2538 - accuracy: 0.5463
Epoch 4/40
391/391 [==============================] - 16s 40ms/step - loss: 1.1420 - accuracy: 0.5948
Epoch 5/40
391/391 [==============================] - 15s 40ms/step - loss: 1.0619 - accuracy: 0.6254
Epoch 6/40
391/391 [==============================] - 16s 40ms/step - loss: 0.9769 - accuracy: 0.6559
Epoch 7/40
391/391 [==============================] - 15s 39ms/step - loss: 0.9182 - accuracy: 0.6778
Epoch 8/40
391/391 [==============================] - 15s 38ms/step - loss: 0.8601 - accuracy: 0.6977
Epoch 9/40
391/391 [==============================] - 15s 38ms/step - loss: 0.8148 - accuracy: 0.7126
Epoch 10/40
391/391 [==============================] - 15s 38ms/step - loss: 0.761

We zien dat hier een hoge accuracy gehaald wordt. Het is dus nu tijd om het op de testset uit te voeren om de uiteindelijke accuracy te berekenen.

## Testing the Model

In [2]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

NameError: name 'model' is not defined